# 🎯 RAFT Dataset Creation

This notebook creates the final RAFT (Retrieval Augmented Fine-Tuning) dataset.

**RAFT Format:**
- Each example has 1 oracle document (contains answer) + 4 distractor documents
- 80% examples: oracle + distractors
- 20% examples: only distractors (teach model to say "I don't know")

**Input:** `qa_pairs.jsonl`, `chunks.jsonl`
**Output:** `train.jsonl`, `val.jsonl`

## 1. Setup

In [ ]:
# Install dependencies (run once)
# For local setup, use pip in terminal:
# pip install sentence-transformers faiss-cpu tqdm torch

# Or uncomment below to install in notebook:
# !uv pip install -q sentence-transformers faiss-cpu tqdm torch

In [ ]:
# ===== LOCAL SETUP (Google Drive Desktop) =====
import os

# Google Drive Desktop path
DATA_DIR = "G:/My Drive/RAFT_dental_data"

CHUNKS_FILE = f"{DATA_DIR}/chunks.jsonl"
QA_FILE = f"{DATA_DIR}/qa_pairs.jsonl"
OUTPUT_DIR = f"{DATA_DIR}/raft_dataset"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Verify inputs
print(f"Data directory: {DATA_DIR}")
for f in [CHUNKS_FILE, QA_FILE]:
    if os.path.exists(f):
        size_mb = os.path.getsize(f) / (1024**2)
        print(f"✓ Found {os.path.basename(f)} ({size_mb:.1f} MB)")
    else:
        print(f"✗ Missing {os.path.basename(f)} - check your path!")

## 2. Load Data

In [ ]:
import json
import random
from tqdm.notebook import tqdm

# Load chunks (these are smaller, ~100MB)
chunks = []
chunk_by_id = {}
with open(CHUNKS_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        chunk = json.loads(line)
        chunks.append(chunk)
        chunk_by_id[chunk['chunk_id']] = chunk

print(f"✓ Loaded {len(chunks)} chunks")

# Count Q&A pairs without loading into memory
print("Counting Q&A pairs...")
qa_count = 0
with open(QA_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        qa_count += 1
print(f"✓ Found {qa_count} Q&A pairs (will stream during processing)")

## 3. Create Embedding Index for Distractor Selection

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import torch

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Use FASTER embedding model (still good quality, 5x faster)
# Options:
# - 'all-MiniLM-L6-v2' = FASTEST (384D, great quality)
# - 'pritamdeka/S-PubMedBert-MS-MARCO' = Medical domain but slower

print("Loading embedding model...")
embed_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
# embed_model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO', device=device)  # Use this for medical domain

print(f"✓ Model loaded: {embed_model.get_sentence_embedding_dimension()}D embeddings")

In [ ]:
# Generate embeddings - MAXIMUM SPEED
print("Generating chunk embeddings (MAX GPU speed)...")

chunk_texts = [c['text'] for c in chunks]

# Maximum speed settings
import torch
with torch.cuda.amp.autocast():  # Use fp16 for speed
    chunk_embeddings = embed_model.encode(
        chunk_texts,
        show_progress_bar=True,
        batch_size=512,  # Even larger batch
        convert_to_numpy=True,
        normalize_embeddings=True,
        device='cuda'
    )

print(f"✓ Embeddings shape: {chunk_embeddings.shape}")

In [ ]:
# Build FAISS index (embeddings already normalized)
print("Building FAISS index...")

dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner product = cosine for normalized vectors
index.add(chunk_embeddings.astype(np.float32))

print(f"✓ FAISS index built with {index.ntotal} vectors")

## 4. RAFT Dataset Builder

In [ ]:
from typing import List, Dict, Optional, Iterator
from dataclasses import dataclass
import numpy as np

@dataclass
class RAFTConfig:
    """Configuration for RAFT dataset creation."""
    num_distractors: int = 4
    oracle_ratio: float = 0.8
    max_context_length: int = 3000
    train_ratio: float = 0.9


class RAFTDatasetBuilder:
    """Build RAFT dataset with SEMANTIC distractors - OPTIMIZED."""
    
    def __init__(
        self,
        chunks: List[Dict],
        chunk_embeddings: np.ndarray,
        faiss_index: faiss.Index,
        embed_model: SentenceTransformer,
        config: RAFTConfig = None
    ):
        self.chunks = chunks
        self.chunk_embeddings = chunk_embeddings
        self.index = faiss_index
        self.embed_model = embed_model
        self.config = config or RAFTConfig()
        self.chunk_by_id = {c['chunk_id']: i for i, c in enumerate(chunks)}
    
    def find_distractors_batch(self, questions: List[str], oracle_indices: List[int], k: int = 6) -> List[List[int]]:
        """Find semantic distractors for batch of questions."""
        # Batch encode on GPU
        q_embeddings = self.embed_model.encode(
            questions, 
            convert_to_numpy=True, 
            show_progress_bar=False,
            batch_size=len(questions)  # Process all at once
        )
        faiss.normalize_L2(q_embeddings)
        
        # Batch FAISS search
        _, all_indices = self.index.search(q_embeddings.astype(np.float32), k + 5)
        
        results = []
        for i, indices in enumerate(all_indices):
            oracle_idx = oracle_indices[i]
            distractors = [int(idx) for idx in indices if idx != oracle_idx]
            results.append(distractors[:k])
        return results
    
    def format_context_document(self, chunk: Dict, is_oracle: bool = False) -> Dict:
        return {
            "content": chunk['text'],
            "source": chunk.get('source', 'Unknown'),
            "page_number": chunk.get('page_number', 0),
            "category": chunk.get('category', 'Unknown'),
            "is_oracle": is_oracle
        }
    
    def _create_no_oracle_answer(self, question: str) -> str:
        return (
            "Based on the provided documents, I cannot find sufficient information "
            "to accurately answer this question. The documents do not contain "
            "relevant content about this specific topic."
        )
    
    def stream_qa_pairs(self, qa_file: str) -> Iterator[Dict]:
        with open(qa_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    yield json.loads(line)
                except:
                    continue
    
    def build_dataset_streaming(
        self,
        qa_file: str,
        output_dir: str,
        total_count: int,
        batch_size: int = 1000
    ) -> Dict[str, int]:
        """Build RAFT dataset with semantic distractors - OPTIMIZED batching."""
        print(f"Building RAFT dataset from {total_count} Q&A pairs (semantic distractors)...")
        print(f"Batch size: {batch_size}")
        
        train_path = os.path.join(output_dir, "train.jsonl")
        val_path = os.path.join(output_dir, "val.jsonl")
        
        total_examples = 0
        failed = 0
        with_oracle_count = 0
        without_oracle_count = 0
        
        split_idx = int(total_count * self.config.train_ratio)
        
        train_f = open(train_path, 'w', encoding='utf-8')
        val_f = open(val_path, 'w', encoding='utf-8')
        
        try:
            batch = []
            global_idx = 0
            
            for qa in tqdm(self.stream_qa_pairs(qa_file), total=total_count, desc="Processing"):
                batch.append((qa, global_idx))
                global_idx += 1
                
                if len(batch) >= batch_size:
                    # Process batch
                    results = self._process_batch_semantic(batch, split_idx)
                    
                    for example, is_train, success, has_oracle in results:
                        if success:
                            line = json.dumps(example, ensure_ascii=False) + "\n"
                            if is_train:
                                train_f.write(line)
                            else:
                                val_f.write(line)
                            total_examples += 1
                            if has_oracle:
                                with_oracle_count += 1
                            else:
                                without_oracle_count += 1
                        else:
                            failed += 1
                    
                    batch = []
                    
                    if global_idx % 20000 == 0:
                        train_f.flush()
                        val_f.flush()
            
            # Process remaining
            if batch:
                results = self._process_batch_semantic(batch, split_idx)
                for example, is_train, success, has_oracle in results:
                    if success:
                        line = json.dumps(example, ensure_ascii=False) + "\n"
                        if is_train:
                            train_f.write(line)
                        else:
                            val_f.write(line)
                        total_examples += 1
                        if has_oracle:
                            with_oracle_count += 1
                        else:
                            without_oracle_count += 1
                    else:
                        failed += 1
        
        finally:
            train_f.close()
            val_f.close()
        
        train_count = min(split_idx, total_examples)
        val_count = max(0, total_examples - train_count)
        
        print(f"\n✓ Created {total_examples} examples ({failed} failed)")
        print(f"✓ Train: {train_count} | Val: {val_count}")
        
        return {
            "total": total_examples,
            "train": train_count,
            "val": val_count,
            "failed": failed,
            "with_oracle": with_oracle_count,
            "without_oracle": without_oracle_count
        }
    
    def _process_batch_semantic(self, batch: List, split_idx: int) -> List:
        """Process batch with semantic distractor search."""
        # Collect valid items
        valid_items = []
        for qa, global_idx in batch:
            question = qa.get('question', '')
            answer = qa.get('answer', '')
            chunk_id = qa.get('chunk_id', '')
            
            if not question or not answer:
                continue
            
            oracle_idx = self.chunk_by_id.get(chunk_id, -1)
            if oracle_idx == -1:
                continue
            
            valid_items.append((qa, oracle_idx, global_idx))
        
        if not valid_items:
            return [(None, False, False, False) for _ in batch]
        
        # Batch find distractors (ONE embedding call for entire batch)
        questions = [item[0]['question'] for item in valid_items]
        oracle_indices = [item[1] for item in valid_items]
        all_distractors = self.find_distractors_batch(questions, oracle_indices, k=self.config.num_distractors + 2)
        
        # Build results
        results = []
        valid_map = {item[2]: i for i, item in enumerate(valid_items)}  # global_idx -> valid_idx
        
        for qa, global_idx in batch:
            if global_idx not in valid_map:
                results.append((None, global_idx < split_idx, False, False))
                continue
            
            valid_idx = valid_map[global_idx]
            oracle_idx = valid_items[valid_idx][1]
            distractor_indices = all_distractors[valid_idx]
            
            include_oracle = random.random() < self.config.oracle_ratio
            
            context = []
            if include_oracle:
                context.append(self.format_context_document(self.chunks[oracle_idx], is_oracle=True))
                for idx in distractor_indices[:self.config.num_distractors]:
                    context.append(self.format_context_document(self.chunks[idx], is_oracle=False))
                final_answer = qa.get('answer', '')
                with_oracle_count = True
            else:
                for idx in distractor_indices[:self.config.num_distractors + 1]:
                    context.append(self.format_context_document(self.chunks[idx], is_oracle=False))
                final_answer = self._create_no_oracle_answer(qa['question'])
                with_oracle_count = False
            
            random.shuffle(context)
            
            example = {
                "question": qa['question'],
                "context": context,
                "answer": final_answer,
                "has_oracle": include_oracle,
                "category": qa.get('category', 'Unknown'),
                "difficulty": qa.get('difficulty', 'medium'),
                "question_type": qa.get('type', 'unknown')
            }
            
            results.append((example, global_idx < split_idx, True, include_oracle))
        
        return results

## 5. Build RAFT Dataset

In [ ]:
# Configure RAFT
config = RAFTConfig(
    num_distractors=4,      # 4 distractor docs per example
    oracle_ratio=0.8,       # 80% have oracle, 20% only distractors
    train_ratio=0.9         # 90% train, 10% val
)

print("RAFT Configuration:")
print(f"  Distractors per example: {config.num_distractors}")
print(f"  Oracle ratio: {config.oracle_ratio}")
print(f"  Train/Val split: {config.train_ratio}/{1-config.train_ratio}")

In [ ]:
# Build dataset with SEMANTIC distractors (proper quality)
builder = RAFTDatasetBuilder(
    chunks=chunks,
    chunk_embeddings=chunk_embeddings,
    faiss_index=index,
    embed_model=embed_model,
    config=config
)

# batch_size=1000 is the sweet spot for speed vs memory
stats = builder.build_dataset_streaming(QA_FILE, OUTPUT_DIR, qa_count, batch_size=1000)

## 6. Validate Dataset

In [ ]:
# Load and inspect train set
train_path = os.path.join(OUTPUT_DIR, "train.jsonl")
train_examples = []
with open(train_path, 'r', encoding='utf-8') as f:
    for line in f:
        train_examples.append(json.loads(line))

print(f"Train examples: {len(train_examples)}")

# Sample example
example = train_examples[0]
print("\nSample RAFT example:")
print(f"Question: {example['question']}")
print(f"Has Oracle: {example['has_oracle']}")
print(f"Context documents: {len(example['context'])}")
print(f"Answer preview: {example['answer'][:200]}...")

In [ ]:
# Show context structure
print("\nContext documents:")
for i, doc in enumerate(example['context']):
    print(f"\nDoc {i+1}:")
    print(f"  Source: {doc['source']}")
    print(f"  Page: {doc['page_number']}")
    print(f"  Is Oracle: {doc['is_oracle']}")
    print(f"  Content: {doc['content'][:100]}...")

In [ ]:
# Validate no-oracle examples
no_oracle = [e for e in train_examples if not e['has_oracle']]
print(f"\nNo-oracle examples: {len(no_oracle)}")

if no_oracle:
    sample = no_oracle[0]
    print(f"\nSample no-oracle example:")
    print(f"Question: {sample['question']}")
    print(f"Answer: {sample['answer']}")
    print(f"All is_oracle values: {[d['is_oracle'] for d in sample['context']]}")

## 7. Summary

In [ ]:
train_size = os.path.getsize(train_path) / (1024**2)
val_path = os.path.join(OUTPUT_DIR, "val.jsonl")
val_size = os.path.getsize(val_path) / (1024**2)

print("="*60)
print("RAFT DATASET CREATION COMPLETE")
print("="*60)
print(f"📊 Total examples: {stats['total']}")
print(f"📈 Train examples: {stats['train']} ({train_size:.1f} MB)")
print(f"📉 Val examples: {stats['val']} ({val_size:.1f} MB)")
print(f"✅ With oracle: {stats['with_oracle']} ({100*stats['with_oracle']/stats['total']:.1f}%)")
print(f"❌ Without oracle: {stats['without_oracle']} ({100*stats['without_oracle']/stats['total']:.1f}%)")
print(f"⚠️ Failed: {stats['failed']}")
print(f"📁 Output: {OUTPUT_DIR}")
print("="*60)
print("Next: Run 04_embedding_creation.ipynb (for FAISS index)")
print("Then: Run 05_model_training.ipynb (for QLoRA fine-tuning)")
print("="*60)